In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

In [ ]:
sd_hf["transformer.wpe.weight"].view(-1)[:20]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(sd_hf["transformer.wpe.weight"], cmap="gray")

In [ ]:
plt.plot(sd_hf["transformer.wpe.weight"][:, 150])
plt.plot(sd_hf["transformer.wpe.weight"][:, 200])
plt.plot(sd_hf["transformer.wpe.weight"][:, 250])

In [ ]:
plt.imshow(sd_hf["transformer.h.1.attn.c_attn.weight"][:300,:300], cmap="gray")

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

In [ ]:
from transformers import GPT2Tokenizer
from train_gpt2 import GPT
import torch
import torch.nn.functional as F
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
prompt = "Hello, I'm a language model,"
tokens = torch.tensor(tokenizer.encode(prompt))

model = GPT.from_pretrained("gpt2")
model.eval()
model.to("cpu")


In [ ]:
num_return_sequences = 5
max_length = 30
with torch.no_grad():
    sample_tokens = tokens.repeat(num_return_sequences, 1)
    for _ in range(max_length):
        logits = model(sample_tokens)
        logits, top_indices = torch.topk(logits[:,-1,:], k=100, dim=-1)
        probs = F.softmax(logits, dim=-1)
        sampled_indices = torch.multinomial(probs, num_samples=1)  
        next_token_indices = torch.gather(top_indices, dim=1, index=sampled_indices)
        sample_tokens = torch.cat([sample_tokens, next_token_indices], 1)
        
    for sample_token in sample_tokens:
        print(tokenizer.decode(sample_token))
        print("-"*40)        


In [ ]:
top_indices